## 1. Introduction
<p>In this notebook, I parsed the dataset, cleaned up partial data, and generated two synthesized datasets which were exported as two csv files: unique_id_records_dataset.csv and aggregate_by_month_dataset.csv.</p>

In [ ]:
import csv
from datetime import datetime

import pandas as pd

## 2. Reading From The Raw CSV File
<p>Initially, I parsed raw event log.csv to create a dataset that has 1 record for each unique user ID, with the following columns: date of activation, total logins, total vault events, and time to activate. As time to activate is calculated with the creation date and date of activation, I created an extra column to store the creation date data.</p>


In [ ]:
def convert_string_to_datetime(date_str: str):
    datetime_object = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    return datetime_object


def parse_unique_id_data(file_path: str):
    table_rows = {}
    with open(file_path, 'r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            if row[2] == 'create':
                table_rows[row[0]] = [None, 0, 0, None, convert_string_to_datetime(row[1])]
            if row[2] == 'activate':
                if row[0] in table_rows:
                    table_rows[row[0]][0] = convert_string_to_datetime(row[1])
                    table_rows[row[0]][3] = table_rows[row[0]][0] - table_rows[row[0]][4]
            if row[2] == 'login':
                table_rows[row[0]][1] += 1
            if row[2] == 'add_to_vault':
                table_rows[row[0]][2] += 1
    return table_rows

parsed_rows = parse_unique_id_data("raw_event_log.csv")

## 3. Removing Extra Column From Unique Id Data
<p>I removed the extra creation date column from the unique id dataset.</p>

In [ ]:
def remove_creation_date_from_unique_id_data(table_rows: dict):
    for key in table_rows:
        table_rows[key] = table_rows[key][:-1]
    return table_rows

all_rows = remove_creation_date_from_unique_id_data(parsed_rows)

## 4. Prep The Unique Id Data
<p>I prepared the data by reformatting the date for the date of activation and the time in seconds for the time to activate.</p>

In [ ]:
def prep_unique_id_data_row(row_list: list):
    if row_list[0] is not None:
        row_list[0] = row_list[0].strftime('%Y-%m-%d %H:%M:%S')
    if row_list[3] is not None:
        row_list[3] = int(row_list[3].total_seconds())
    return row_list


## 5. Write The Unique Id Data To A CSV File
<p>I wrote the unique id data to the dataset_unique_id_records.csv file.</p>

In [ ]:
def write_unique_id_data_to_file(table_header: tuple, table_rows: dict, file_path: str):
    with open(file_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(table_header)
        for tbl_row in table_rows:
            cleaned_row = prep_unique_id_data_row(table_rows[tbl_row])
            writer.writerow((tbl_row,
                             cleaned_row[0],
                             cleaned_row[1],
                             cleaned_row[2],
                             cleaned_row[3]))

all_headers = "user", "date_of_activation", "total_logins", "total_vault_events", \
              "time_to_activate"
write_unique_id_data_to_file(all_headers, all_rows, 'dataset_unique_id_records.csv')

## 6. Create A DataFrame For The Aggregate By Month Data
<p>I created a dataframe using the all_headers and all_rows data from the unique id dataset.</p>

In [ ]:
df_unique_id_data = pd.DataFrame.from_dict(all_rows, orient='index', columns=all_headers[1:]).astype(
        {'date_of_activation': 'datetime64'})

## 7. Prep The Aggregate By Month Data
<p>I prepped the aggregate by month data by changing seconds to date, removing unnecessary columns, grouping the data by month and year, and renaming the appropriate columns.</p>

In [ ]:
def convert_seconds_to_days(seconds_col: pd.DataFrame):
    seconds_col = pd.to_timedelta(seconds_col, unit="s")
    day_duration = pd.to_timedelta(1, unit='D')
    days = seconds_col / day_duration
    return days


def prep_aggregate_by_month_data(dataframe: pd.DataFrame):
    dataframe.reset_index(drop=True, inplace=True)
    dataframe = dataframe[["date_of_activation", "time_to_activate", "total_logins"]].copy()
    dataframe['time_to_activate'] = convert_seconds_to_days(dataframe['time_to_activate'])
    dataframe.index = dataframe["date_of_activation"]
    dataframe = dataframe.groupby(dataframe["date_of_activation"].dt.to_period("M")).mean()
    dataframe = dataframe.rename_axis("activation_month").rename(
        columns={"time_to_activate": "average_days_to_activate",
                 "total_logins": "average_logins"})
    return dataframe

df_aggregate_by_month_data = prep_aggregate_by_month_data(df_unique_id_data)

## 8. Write The Aggregate By Month Data to a CSV File
<p>I wrote the aggregate by month data to the dataset_aggregate_by_month.csv file.</p>

In [ ]:
df_aggregate_by_month_data.to_csv('dataset_aggregate_by_month.csv', index=True)